In [1]:
import keras
import numpy as np

In [2]:
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

600901/600901 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
Corpus length: 600893


In [3]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 200278


In [8]:
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Unique characters: 57
Vectorization...


In [10]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
import random
import sys
for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(X, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)

epoch 1
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 2.3194
--- Generating with seed: " forms of nature, with fearless
oedipus-eyes, and stopped ul"
------ temperature: 0.2
 forms of nature, with fearless
oedipus-eyes, and stopped ul------ temperature: 0.5
 forms of nature, with fearless
oedipus-eyes, and stopped ul------ temperature: 1.0
 forms of nature, with fearless
oedipus-eyes, and stopped ul------ temperature: 1.2
 forms of nature, with fearless
oedipus-eyes, and stopped ulastiat
of impenine"", in a will
abseudumpuamitif--in
ligenne
solaces whas  so
"one pays,ith abool the lofe for mentally, butucam bhom
uniftes in
onrbegully, woeld,    thong,
best in which on erean the all bad the a
jlerman, woaks imsuscize
such madowëhk grog.w---has oprilsly, reased i  pociaius,"flivicl.--ethrs" "this theyseli,e be w"ble untt crile,, that
whether it frait insacrite; bysinds, whothepoch 2
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 1.6658
--- Generating with seed: " of himself. ev